# Fine-tuning Otimizado de GPT-2 com PEFT (LoRA) no Dataset AmazonTitles-1.3MM

Este notebook demonstra um processo otimizado de fine-tuning do modelo GPT-2 usando PEFT com LoRA, visando reduzir significativamente o tempo de treinamento.

In [ ]:
!pip install transformers datasets torch peft accelerate

In [ ]:
import torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from accelerate import Accelerator

In [ ]:
# Carregando uma fração do dataset
dataset = load_dataset('json', data_files={'train': 'path/to/your/dataset/trn.json'})
dataset = dataset['train'].shuffle(seed=42).select(range(100000))  # Usando apenas 100k amostras

def prepare_data(examples):
    return {
        'text': [f"Title: {title}\nContent: {content}\nResponse:" for title, content in zip(examples['title'], examples['content'])]
    }

dataset = dataset.map(prepare_data, batched=True, remove_columns=dataset.column_names)
dataset = dataset.train_test_split(test_size=0.1)

print(f"Tamanho do conjunto de treinamento: {len(dataset['train'])}")
print(f"Tamanho do conjunto de validação: {len(dataset['test'])}")

In [ ]:
model_name = 'gpt2'  # Usando o modelo menor 'gpt2' em vez de 'gpt2-large'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)  # Reduzindo max_length

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Configuração otimizada do LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=4,  # Reduzindo o rank
    lora_alpha=16,
    lora_dropout=0.05
)

model = get_peft_model(model, peft_config)

In [ ]:
# Configuração otimizada do treinamento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Reduzindo o número de épocas
    per_device_train_batch_size=32,  # Aumentando o tamanho do batch
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    fp16=True,  # Habilitando treinamento em precisão mista
    gradient_accumulation_steps=2,
    learning_rate=5e-4,  # Aumentando a taxa de aprendizado
)

# Inicializando o Accelerator
accelerator = Accelerator()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
)

In [ ]:
# Treinamento
trainer.train()

In [ ]:
# Salvando o modelo
peft_model_path = "./peft_model_optimized"
trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

from google.colab import files
!zip -r peft_model_optimized.zip peft_model_optimized
files.download('peft_model_optimized.zip')

In [ ]:
# Função para gerar respostas (sem alterações)
def generate_response(title, content):
    input_text = f"Title: {title}\nContent: {content}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    
    with torch.no_grad():
        output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    response = generated_text.split('Response:')[-1].strip()
    
    return response

# Exemplo de uso
title = "Smartphone de última geração"
content = "Este smartphone possui câmera de alta resolução, bateria de longa duração e processador potente."
response = generate_response(title, content)
print(f"Título: {title}")
print(f"Conteúdo: {content}")
print(f"Resposta gerada: {response}")